In [3]:
f_path = 'C:\\Users\\Ali Salem\\Desktop\\App_Update\\static/files\\parameters.xlsx'
slides_name = ['SKU Profitability', 'Trade Margin Table By Sector', 'Trade Margin Table By Segment', 'Trade Margin Table By Subsegment', 'Trade Margin Table By Subcategory']

In [4]:
import os
%run "{os.path.dirname(os.getcwd())}\general_functions\generalFunctions.ipynb" #container

%run "{os.path.dirname(os.getcwd())}\general_functions\Extracting Data Functions.ipynb" # container

In [5]:
filename = 'parameters.xlsx'

# Get the current working directory
current_dir = os.path.dirname(os.getcwd())

# Construct the full path to the file
f_path = os.path.join(current_dir, filename)
print(f_path)
#xls = pd.ExcelFile(f_path)
parm = pd.read_excel(f_path, sheet_name='Financials')
fields = dict(zip(parm['Field'],parm['Value']))

c:\Users\Ali Salem\Desktop\App_Update\parameters.xlsx


In [6]:
import time
import adodbapi
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

In [7]:
start_time = time.time()

In [ ]:
server = "powerbi://api.powerbi.com/v1.0/myorg/"+ fields['server']
dataset_name = fields['f_name']
f_name = os.getcwd()+"/"+fields['f_name']+".xlsx"

client_manuf = list(set(fields['client_manuf'].split(','))-set(['']))
client_brands = list(set(fields['client_brands'].split(','))-set(['']))

ManufOrTopC = fields['ManufOrTopC']
BrandOrTopB = fields['BrandOrTopB']

decimals = fields['decimals']
sign = fields['sign']
currency = fields['currency']
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' ' 

prodORitem=fields["prodORitem"]
categories = list(set(fields['categories'].split(','))-set(['']))
sectors=list(set(fields['sectors'].split(','))-set(['']))
segments=list(set(fields['segments'].split(','))-set(['']))
subsegments=list(set(fields['subsegments'].split(','))-set(['']))
subcategories=list(set(fields['subcategories'].split(','))-set(['']))

national=fields['national']
customareas=fields['customareas']
print(fields['customareas'])
if pd.isna(customareas):
    customareas = ''
areas = list(set(fields['areas'].split(','))-set(['']))+[customareas]
areas = [a for a in areas if a != '']

regions_RET = list(set(fields['regions_RET'].split(','))-set(['']))
channels_RET = list(set(fields['channels_RET'].split(','))-set(['']))
market_RET=list(set(fields['market_RET'].split(','))-set(['']))

regions_CHAN=list(set(fields['regions_CHAN'].split(','))-set(['']))
channels_CHAN=list(set(fields['channels_CHAN'].split(','))-set(['']))
market_CHAN=list(set(fields['market_CHAN'].split(','))-set(['']))

regions_CUST=list(set(fields['regions_CUST'].split(','))-set(['']))
channels_CUST=list(set(fields['channels_CUST'].split(','))-set(['']))
market_CUST=list(set(fields['market_CUST'].split(','))-set(['']))

data_source=fields['data_source']

years=list(set(fields['years'].split(','))-set(['']))
years = {int(y) for y in fields['years'].split(',') if y}
end_date=fields['end_date']

brands_only=fields['brands_only']
National=["NATIONAL"]if national else []


client_brands,ManufOrTopC,decimals,sign,currency,prodORitem,categories

REGIONS


(['Schick', 'Cremo', 'Equate'],
 'Top Companies',
 2,
 'Before',
 '$ ',
 'SKU',
 ['Manual Shave Men'])

In [11]:
regions = regions_RET + regions_CHAN + regions_CUST
channels = channels_RET + channels_CHAN + channels_CUST
markets = market_RET + market_CHAN + market_CUST

entity_hierarchy = [
    ("Area",National),
    ("Region", regions),
    ("Channel", channels),
    ("Market", markets)
]
hierarchy_levels = [
    ("Category", categories),
    ("Sector", sectors),
    ("Segment", segments),
    ("SubSegment", subsegments),
    ("SubCategory", subcategories)
]

direct_parent = {"Sector":list(zip(["Category"]*len(categories),categories))
                ,"Segment":list(zip(["Sector"]*len(sectors),sectors))
                #,"SubSegment":list(zip(["Segment"]*len(sectors),sectors))
                #,"SubCategory":list(zip(["Segment"]*len(segments),segments))
                }


In [13]:
conn_str = f"Provider=MSOLAP.8;Data Source={server};Initial Catalog={dataset_name};"
path=os.path.join(os.getcwd(),"Financials Datasets Test")

past_12_months = pd.date_range(end=end_date, periods=12, freq='ME').strftime('%b-%y').tolist()
past_3_months = pd.date_range(end=end_date, periods=3, freq='ME').strftime('%b-%y').tolist()
p12m_dax = "{" + ", ".join(f'"{date}"' for date in past_12_months) + "}"
p3m_dax = "{" + ", ".join(f'"{date}"' for date in past_3_months) + "}"


### Slide 67(brands_client_dfs)

In [14]:

def execute_dax_query(BrandorManuf,entity_name, area, hierby, entity_type):
    outputdic = {}
    columns=["Gross Margin %","Gross Margin % IYA","Gross Profit","Net Sales","Net Sales/Kg","Net Sales/Kg IYA","Volume Sales","Trade Margin %","Trade Profit","Trade Profit/Kg","Av Price/KG","Value Sales IYA","Value Sales","Total WoB %"]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        SUMMARIZE(
            Products,
            Products[{BrandorManuf}],  
            {column_exprs}
        ),
        Products[Category] = "{categories[0]}",
        Products[{ManufOrTopC}] = "{client_manuf[0]}",
        
        Products[{hierby}] = "{entity_type}",
        TREATAS(
            {p12m_dax},
            Calendar[MonthYear]
        ),
        TREATAS(
            {{"{entity_name}"}},
            Market[{area}]
        ),
        FILTER(
            'Scope', 
            'Scope'[Scope] = "{hierby}"   
        )
    )   
    """
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[{hierby}]),  -- dummy base just to return a single row
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            Products[{ManufOrTopC}] = "{client_manuf[0]}",
            
            Products[{hierby}] = "{entity_type}",
            TREATAS(
                {p12m_dax},
                Calendar[MonthYear] 
            ),
            TREATAS(
                {{"{entity_name}"}},
                Market[{area}]
            ),
            FILTER(
                'Scope', 
                'Scope'[Scope] = "{hierby}"   
            )
        )   
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
                
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()        
                    
            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
        
            dt = pd.DataFrame([grandtotal_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in grandtotal_columns])
            dt[df.columns[0]] = 'Grand Total'            
            df = pd.concat([df, dt], ignore_index=True)
            df = df.iloc[:, :-1]

            
            outputdic[f"{client_manuf[0]} | {entity_type} | {entity_name}"] = df
            print(f"Query executed successfully for {entity_type} {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_type} {entity_name}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_type} {entity_name}: {e}")
    return outputdic

# List of all entities to process

brands_client_dfs = {}
# **Threaded Execution**
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = []
    for area, entity_list in entity_hierarchy:
        for entity in entity_list:
            for hierby, hier_values in hierarchy_levels:
                    for value in hier_values:
                        futures.append(executor.submit(execute_dax_query, f"{BrandOrTopB}",entity, area, hierby, value))
               
    # Wait for all tasks to complete
    for future in futures:
        result=future.result()
        brands_client_dfs.update(result)

pd.to_pickle(brands_client_dfs, os.path.join(path,"brands_client_dfs.pkl"))


Query executed successfully for Manual Shave Men Sam's Corp.
Query executed successfully for Manual Shave Men NATIONAL.
Query executed successfully for Razors NATIONAL.
Query executed successfully for Refills NATIONAL.
Query executed successfully for System NATIONAL.
Query executed successfully for Disposables NATIONAL.
Query executed successfully for System Sam's Corp.
Query executed successfully for Disposables NATIONAL.
Query executed successfully for Disposables Sam's Corp.
Query executed successfully for Razors Sam's Corp.
Query executed successfully for System Bj's Corp.
Query executed successfully for Disposables Sam's Corp.
Query executed successfully for Refills Sam's Corp.
Query executed successfully for Manual Shave Men Bj's Corp.
Query executed successfully for Disposables Bj's Corp.
Query executed successfully for Razors Bj's Corp.
Query executed successfully for Refills Bj's Corp.
Query executed successfully for Disposables Bj's Corp.
Query executed successfully for Manua

### Slide 68 & 69 & 71 & 72

In [15]:

def execute_dax_query(BrandorManuf,entity_name, area, hierby, entity_type,Monthyear,ke):
    outputdic = {}

    columns=["Value Sales","Value Sales IYA","Trade Margin %","Trade Margin % DYA","Net Sales","Net Sales/Kg","Gross Profit/Kg","Gross Margin %","Trade Profit IYA","VSOD IYA","Av Price/KG","VAT/Kg","Total COGS/Kg","Trade Profit/Kg","Rate of Sales","WD","Total WoB %"]

    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        SUMMARIZE(
            Products,
            Products[{ke}], 
            {column_exprs}            
        ),
        Products[Category] = "{categories[0]}",
        Products[{ManufOrTopC}] = "{client_manuf[0]}",

        TREATAS(
            {Monthyear},
            Calendar[MonthYear]
        ),
        TREATAS(
            {{"{entity_name}"}},
            Market[{area}]
        ),
        FILTER(
            'Scope', 
            'Scope'[Scope] = "Category"   
        )
    )   
    """
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),  -- dummy base just to return a single row
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            Products[{ManufOrTopC}] = "{client_manuf[0]}",
            TREATAS(
                {Monthyear},
                Calendar[MonthYear]
            ),
            TREATAS(
                {{"{entity_name}"}},
                Market[{area}]
            ),
            FILTER(
                'Scope', 
                'Scope'[Scope] = "Category"   
            )
        )   
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
                
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()        
                    
            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
        
            dt = pd.DataFrame([grandtotal_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in grandtotal_columns])
            dt[df.columns[0]] = 'Grand Total'            
            df = pd.concat([df, dt], ignore_index=True)
            df = df.iloc[:, :-1]

            
            outputdic[f"{categories[0]} | {client_manuf[0]} | {entity_name}"] = df
            print(f"Query executed successfully for {entity_type} {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_type} {entity_name}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_type} {entity_name}: {e}")
    return outputdic

# List of all entities to process

brands_client_dfs = {}
def process_dax_queries(entity_hierarchy, hierarchy_levels,Monthyear):
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        dfs_results = {} 
        futures = {}
        ordered_keys=[]
        for ke, val in direct_parent.items():
            for hierby, hier_values in hierarchy_levels:
                for value in hier_values:
                    for area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            key =  f"{client_manuf[0]} | {value} | {entity}"
                            ordered_keys.append(key) 
                            future = executor.submit(execute_dax_query, f"{BrandOrTopB}", entity, area, hierby, value,Monthyear,ke)
                            futures[future] = key


            temp_results = {}
            for future in as_completed(futures):
                key = futures[future]
                result = future.result()
                dfs_results.update(result)


            # Insert results in original order
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]
            if  Monthyear== p12m_dax:
                filename =  f"{ke}_P12M_client_dfs.pkl"
            else:
                filename =  f"{ke}_P3M_client_dfs.pkl"

            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            
            print(f"All DataFrames saved to {output_file}.")


process_dax_queries(entity_hierarchy, hierarchy_levels,p12m_dax)
process_dax_queries(entity_hierarchy, hierarchy_levels,p3m_dax)



Query executed successfully for Manual Shave Men Walmart West.
Query executed successfully for Manual Shave Men Walmart Southwest.
Query executed successfully for Manual Shave Men Walmart Southeast.
Query executed successfully for Manual Shave Men Walmart East.
Query executed successfully for Manual Shave Men Sam's Corp.
Query executed successfully for Manual Shave Men Walmart.
Query executed successfully for Manual Shave Men NATIONAL.
Query executed successfully for Manual Shave Men Bj's Corp.
Query executed successfully for Manual Shave Men Walmart North.
Query executed successfully for Manual Shave Men Walmart Div1 Corp.
Query executed successfully for Manual Shave Men Walmart Nm Corp.
Query executed successfully for Manual Shave Men Walmart Sc Corp.
Query executed successfully for System NATIONAL.
Query executed successfully for System Sam's Corp.
Query executed successfully for System Bj's Corp.
Query executed successfully for System Walmart.
Query executed successfully for System

### Slide 70 &73 &74(Product P12M &P3M)

In [16]:

def execute_dax_query(BrandorManuf,entity_name, area, hierby, entity_type,Monthyear):
    outputdic = {}
    columns=["WD","Gross Profit/Unit","VAT/Unit","Net Sales","Net Sales/Kg","Net Sales/Unit","Gross Margin %","Trade Margin %","Trade Profit/Unit","Av Price/Unit","Total COGS/Unit",f"{prodORitem} Sales Rate","Value Sales IYA"]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        SUMMARIZE(
            Products,
            Products[{BrandorManuf}], 
            Products[{prodORitem}], 
            {column_exprs}            
        ),
        Products[Category] = "{categories[0]}",
        Products[{ManufOrTopC}] = "{client_manuf[0]}",

        Products[{hierby}] = "{entity_type}",
        TREATAS(
            {Monthyear},
            Calendar[MonthYear]
        ),
        TREATAS(
            {{"{entity_name}"}},
            Market[{area}]
        ),
        FILTER(
            'Scope', 
            'Scope'[Scope] = "{hierby}"   
        )
    )   
    """
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[{hierby}]),  -- dummy base just to return a single row
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            Products[{ManufOrTopC}] = "{client_manuf[0]}",
            Products[{hierby}] = "{entity_type}",
            TREATAS(
                {Monthyear},
                Calendar[MonthYear]
            ),
            TREATAS(
                {{"{entity_name}"}},
                Market[{area}]
            ),
            FILTER(
                'Scope', 
                'Scope'[Scope] = "{hierby}"   
            )
        )   
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
                
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()        
                    
            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
        
            dt = pd.DataFrame([grandtotal_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in grandtotal_columns])
            dt[df.columns[0]] = 'Grand Total'            
            df = pd.concat([df, dt], ignore_index=True)
            df = df.iloc[:, :-1]

            
            outputdic[f"{client_manuf[0]} | {entity_type} | {entity_name}"] = df
            print(f"Query executed successfully for {entity_type} {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_type} {entity_name}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_type} {entity_name}: {e}")
    return outputdic

# List of all entities to process

brands_client_dfs = {}
def process_dax_queries(entity_hierarchy, hierarchy_levels,Monthyear):
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        dfs_results = {} 
        futures = {}
        ordered_keys=[]
        for hierby, hier_values in hierarchy_levels:
            for value in hier_values:
                for area, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        key =  f"{client_manuf[0]} | {value} | {entity}"
                        ordered_keys.append(key)  # <-- Add this line
                        future = executor.submit(execute_dax_query, f"{BrandOrTopB}", entity, area, hierby, value,Monthyear)
                        futures[future] = key


        temp_results = {}
        for future in as_completed(futures):
            key = futures[future]
            result = future.result()
            dfs_results.update(result)


        # Insert results in original order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]
        if  Monthyear== p12m_dax:
            filename =  f"product_P12M_dfs.pkl"
        else:
            filename =  f"product_P3M_dfs.pkl"

        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)
        
        print(f"All DataFrames saved to {output_file}.")


process_dax_queries(entity_hierarchy, hierarchy_levels,p12m_dax)
process_dax_queries(entity_hierarchy, hierarchy_levels,p3m_dax)



Query executed successfully for Manual Shave Men Walmart Southwest.
Query executed successfully for Manual Shave Men Walmart East.
Query executed successfully for Manual Shave Men Walmart Southeast.
Query executed successfully for Manual Shave Men Walmart West.
Query executed successfully for Manual Shave Men Walmart.
Query executed successfully for Manual Shave Men NATIONAL.
Query executed successfully for Manual Shave Men Sam's Corp.
Query executed successfully for Manual Shave Men Bj's Corp.
Query executed successfully for Manual Shave Men Walmart North.
Query executed successfully for Manual Shave Men Walmart Div1 Corp.
Query executed successfully for Manual Shave Men Walmart Sc Corp.
Query executed successfully for Manual Shave Men Walmart Nm Corp.
Query executed successfully for System Sam's Corp.
Query executed successfully for System NATIONAL.
Query executed successfully for System Bj's Corp.
Query executed successfully for System Walmart.
Query executed successfully for System

### DataFrames_By Markets

In [17]:
def execute_dax_query(hierby, Monthyear, area, entity_type=None, row_field=None):
    outputdic = {}
    key = f"{client_manuf[0]} | {entity_type}"

    columns = [
        "Gross Margin %", "Gross Margin % IYA", 'Net Sales',
        "Net Sales/Kg", "Net Sales/Kg IYA", "Value Sales IYA"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
    if isinstance(row_field, list):
        row_field_str = ", ".join(col.replace("'", "").replace('"', "") for col in row_field)
    else:
        row_field_str = row_field
    row_field_frist = row_field if isinstance(row_field, str) else row_field[0]
        
    # print(row_field_frist)

    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Market, {row_field_str}),
                {column_exprs}
            ),
            
            TREATAS({Monthyear}, Calendar[MonthYear]),
            FILTER(Products, Products[Category] = "{categories[0]}"),
            FILTER(Products, Products[{ManufOrTopC}] = "{client_manuf[0]}"),
            FILTER(Products, Products[{hierby}] = "{entity_type}"),
            Filter(Market,Market[Area]="{area}"),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    parent_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Market, {row_field_frist}),
                {column_exprs}
            ),
            
            TREATAS({Monthyear}, Calendar[MonthYear]),
            FILTER(Products, Products[Category] = "{categories[0]}"),
            FILTER(Products, Products[{ManufOrTopC}] = "{client_manuf[0]}"),
            FILTER(Products, Products[{hierby}] = "{entity_type}"),
            Filter(Market,Market[Area]="{area}"),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """

    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            Products[{ManufOrTopC}] = "{client_manuf[0]}",
            Products[{hierby}] = "{entity_type}",
            TREATAS({Monthyear}, Calendar[MonthYear]),
            Filter(Market,Market[Area]="{area}"),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            result_columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
        
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(parent_query)
            parent_columns = [desc[0] for desc in cursor.description]
            parenttotal = cursor.fetchall()   
            

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()



        df = pd.DataFrame(data, columns=result_columns)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
        
        parenttotal_df = pd.DataFrame(parenttotal, columns=parent_columns)
        parenttotal_df.columns = parenttotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        parenttotal_df = parenttotal_df.loc[~(parenttotal_df.select_dtypes(include='number') == 0).all(axis=1)]

        if parenttotal_df.shape[1] > 0:
            parenttotal_df.iloc[:, 0] = parenttotal_df.iloc[:, 0].astype(str) +" Total" 
   
        # if parenttotal_df.empty:
        #     outputdic[key] = parenttotal_df
        #     return outputdic
        # if not parenttotal_df.empty:
        #     df = pd.concat([parenttotal_df, df], ignore_index=True)
    

        if grandtotal_data:
            dt = pd.DataFrame([grandtotal_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in grandtotal_columns])
            dt[df.columns[0]] = 'Grand Total'
            df = pd.concat([df, dt], ignore_index=True)
            df = df.iloc[:, :-1]  # Remove duplicate column if needed
       
        df =pd.concat([df,parenttotal_df],ignore_index=True)
        # print(df.columns)
        outputdic[key] = df
        print(f"Query executed successfully for {hierby} | {row_field}.")

    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {hierby} {entity_type} | {row_field}: {db_error}")

    return outputdic


def process_dax_queries(hierarchy_levels, Monthyear, area, row_list):
    with ThreadPoolExecutor(max_workers=8) as executor:
            dfs_results = {}
            futures = [
                executor.submit(execute_dax_query, hierby, Monthyear, area, value, row_list)
                for hierby, hier_values in hierarchy_levels
                for value in hier_values
            ]
                 
            for future in as_completed(futures):
                try:
                    result = future.result()
                    dfs_results.update(result)

                except Exception as e:
                    print(f"Error processing query: {e}")

            if area == "RETAILER":
                if Monthyear==p12m_dax:
                    filename = "retailers_P12M_dfs.pkl"
                else:
                    filename = "retailers_P3M_dfs.pkl"
            elif area == "CHANNEL":
                if Monthyear==p12m_dax:
                    filename = "channel_P12M_dfs.pkl"
                else:    
                    filename = "channel_P3M_dfs.pkl"
                    
            elif area == customareas:
                if Monthyear==p12m_dax:
                    filename = "cust_P12M_dfs.pkl"
                else:    
                    filename = "cust_P3M_dfs.pkl"
                    
            else:
                filename = f"{area.lower()}_dfs.pkl"

            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)

            # print(f"Saved DataFrames for {row_field} to {output_file}.")


# Final Calls
if "RETAILER" in areas:
    RET_list = []
    if regions_RET: RET_list.append('Market[Region]')
    if channels_RET: RET_list.append('Market[Channel]')
    if market_RET: RET_list.append('Market[Market]')
    print(RET_list)
    process_dax_queries(hierarchy_levels, p12m_dax, "RETAILER", RET_list)
    process_dax_queries(hierarchy_levels, p3m_dax, "RETAILER", RET_list)

    

if "CHANNEL" in areas:
    CHA_list = []
    if regions_CHAN: CHA_list.append('Market[Region]')
    if channels_CHAN: CHA_list.append('Market[Channel]')
    if market_CHAN: CHA_list.append('Market[Market]')
    process_dax_queries(hierarchy_levels, p12m_dax, "CHANNEL", CHA_list)
    process_dax_queries(hierarchy_levels, p3m_dax, "CHANNEL", CHA_list)
    

if customareas in areas:
    CUST_list = []
    if regions_CUST: CUST_list.append('Market[Region]')
    if channels_CUST: CUST_list.append('Market[Channel]')
    if market_CUST: CUST_list.append('Market[Market]')
    process_dax_queries(hierarchy_levels, p12m_dax, customareas, CUST_list)
    process_dax_queries(hierarchy_levels, p3m_dax, customareas, CUST_list)
    


['Market[Channel]', 'Market[Market]']


Query executed successfully for Segment | ['Market[Channel]', 'Market[Market]'].
Query executed successfully for Sector | ['Market[Channel]', 'Market[Market]'].
Query executed successfully for Segment | ['Market[Channel]', 'Market[Market]'].
Query executed successfully for Sector | ['Market[Channel]', 'Market[Market]'].
Query executed successfully for Category | ['Market[Channel]', 'Market[Market]'].
Query executed successfully for Segment | ['Market[Channel]', 'Market[Market]'].
Query executed successfully for Segment | ['Market[Channel]', 'Market[Market]'].
Query executed successfully for Segment | ['Market[Channel]', 'Market[Market]'].
Query executed successfully for Sector | ['Market[Channel]', 'Market[Market]'].
Query executed successfully for Segment | ['Market[Channel]', 'Market[Market]'].
Query executed successfully for Category | ['Market[Channel]', 'Market[Market]'].
Query executed successfully for Sector | ['Market[Channel]', 'Market[Market]'].
Query executed successfully fo

### Slide 75(Manuf_dfs)

In [18]:
def execute_dax_query(entity_name, area, hierby, entity_type, ke):
    outputdic = {}
    
    columns = [
        "Volume Sales", "Value Share", "Value Sales", "Value Sales IYA", "Trade Margin %",
        "Net Sales", "Net Sales/Kg", "Gross Profit/Kg", "Gross Margin %", "Gross to Net IYA",
        "VSOD IYA", "Av Price/KG", "VAT/Kg", "Total COGS/Kg", "Trade Profit/Kg",
        "Rate of Sales", "WD", "Total WoB %", "Trade Profit", "Gross Profit"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    base_filters = f"""
        Products[Category] = "{categories[0]}",
        TREATAS({p12m_dax}, Calendar[MonthYear]),
        TREATAS({{ "{entity_name}" }}, Market[{area}]),
        FILTER('Scope', 'Scope'[Scope] = "Category")
    """

    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        SUMMARIZE(
            Products,
            Products[{ManufOrTopC}], 
            Products[{ke}], 
            {column_exprs}
        ),
        {base_filters}
    )
    """

    totalcol_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[{ManufOrTopC}]), 
                {column_exprs}
            ),
            {base_filters}
        )
        """


    grandtotal_query = f"""
    EVALUATE
    CALCULATETABLE(
        ADDCOLUMNS(
            VALUES(Products[Category]),
            {column_exprs}
        ),
        {base_filters}
    )
    """

    try:
        # Main Data
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns_out = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
        df = pd.DataFrame(data, columns=columns_out)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

        # Total by entity
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(totalcol_query)
            totalcol_columns = [desc[0] for desc in cursor.description]
            totalcol_data = cursor.fetchall()
        maintotal_df = pd.DataFrame(totalcol_data, columns=totalcol_columns)
        maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]

        maintotal_df["Top Companies"] = maintotal_df["Top Companies"].astype(str) + " Total"

        # Grand total
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()
        dt = pd.DataFrame([grandtotal_data[0]], columns=[col.replace(']', '').split('[')[-1] for col in grandtotal_columns])
        dt[df.columns[0]] = 'Grand Total'

        # Combine all
        df_with_totals = pd.concat([maintotal_df, dt], ignore_index=True)
        
        df= pd.concat([df, df_with_totals], ignore_index=True)
        df = df.iloc[:, :-1]

        outputdic[f"{categories[0]} | {entity_name}"] = df
        
        print(f"Query executed successfully for {entity_type} {entity_name}.")

    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_type} {entity_name}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_type} {entity_name}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for ke, val in direct_parent.items():
            futures = {}
            ordered_keys = []
            for hierby, hier_values in hierarchy_levels:
                for value in hier_values:
                    for area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            key = f"{categories[0]} | {entity}"
                            ordered_keys.append(key)
                            future = executor.submit(execute_dax_query, entity, area, hierby, value, ke)
                            futures[future] = key

            temp_results = {}
            for future in as_completed(futures):
                key = futures[future]
                try:
                    result = future.result()
                    temp_results.update(result)
                except Exception as e:
                    print(f"Failed to process {key}: {e}")

            dfs_results = {}
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]

            filename = f"manuf_{ke}_dfs"
            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)

            print(f"All DataFrames saved to {output_file}.")
process_dax_queries(entity_hierarchy, hierarchy_levels)


Query executed successfully for Manual Shave Men Walmart Southwest.
Query executed successfully for Manual Shave Men Walmart West.
Query executed successfully for Manual Shave Men Walmart Southeast.
Query executed successfully for Manual Shave Men Walmart East.
Query executed successfully for Manual Shave Men Sam's Corp.
Query executed successfully for Manual Shave Men NATIONAL.
Query executed successfully for Manual Shave Men Walmart.
Query executed successfully for Manual Shave Men Bj's Corp.
Query executed successfully for Manual Shave Men Walmart North.
Query executed successfully for Manual Shave Men Walmart Div1 Corp.
Query executed successfully for Manual Shave Men Walmart Nm Corp.
Query executed successfully for Manual Shave Men Walmart Sc Corp.
Query executed successfully for System NATIONAL.
Query executed successfully for System Sam's Corp.
Query executed successfully for System Bj's Corp.
Query executed successfully for System Walmart.
Query executed successfully for System

In [19]:
loaded_data = {}
datasets_path =os.getcwd()+"\\Financials Datasets Test\\"
datasets = os.listdir(datasets_path)
for d in datasets:
    with open(datasets_path+d, 'rb') as handle:
        globals()[d.split('.')[0]] = pd.read_pickle(handle)

In [20]:
def convert_pickle_to_excel(pickle_path):
    """
    Convert pickle files containing dictionaries of DataFrames to Excel files
    Each key becomes a sheet name in the Excel file
    """
    
    datasets = os.listdir(pickle_path)
    pickle_files = [f for f in datasets if f.endswith('.pkl')]
    
    for pickle_file in pickle_files:
        print(f"Processing {pickle_file}...")
        
        # Load the pickle file
        with open(os.path.join(pickle_path, pickle_file), 'rb') as handle:
            data_dict = pd.read_pickle(handle)
        
        # Get base filename without extension
        base_name = pickle_file.split('.')[0]
        excel_path = os.path.join(pickle_path, f"{base_name}.xlsx")
        
        # Convert to Excel file with multiple sheets
        with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
            for key, df in data_dict.items():
                # Clean sheet name (Excel has 31 character limit and invalid characters)
                sheet_name = key.replace('|', '_').replace(' ', '_').replace('/', '_')
                sheet_name = sheet_name[:31]  # Excel sheet name limit
                
                # Write DataFrame to sheet
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        print(f"  ✓ Created Excel file: {excel_path} with {len(data_dict)} sheets")

# Usage - add this at the very end of your notebook
datasets_path = os.getcwd() + "\\Financials Datasets Test csv\\"
# convert_pickle_to_excel(datasets_path)

print("All pickle files have been converted to Excel files!")

All pickle files have been converted to Excel files!
